In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
from pmi_utils import get_unique_msv
import matplotlib.pyplot as plt
from scipy.stats import norm

### loading data from data folder (for multiple files)

In [2]:
def load_vendor_data(vendor, data_folder):
    vendor_data = pd.DataFrame()
    for file in tqdm(os.listdir(os.path.join(data_folder, vendor))):
        filepath = os.path.join(data_folder, vendor, file)
        vendor_part = pd.read_csv(filepath, index_col=0)
        vendor_part['vendor'] = [vendor] * len(vendor_part)
        vendor_data = vendor_data.append(vendor_part)
    
    return vendor_data

In [3]:
months = {'may': 5, 'june': 6, 'april': 4, 'march': 3}
vendors_df = pd.DataFrame()
important_cols = ['locode', 'phrase', 'average_msv']

for month, month_id in months.items():
    data_folder = f'../data/ua_vendors/{month}'
    month_vendors_data = load_vendor_data('results_27_2020', data_folder)[important_cols]
    month_vendors_data.rename(columns={'average_msv': f'avg_msv_{month}'}, inplace=True)
    if not len(vendors_df):
        vendors_df = month_vendors_data
    else:
        vendors_df = vendors_df.merge(month_vendors_data, on=['locode', 'phrase'])

/Users/new/Dropbox/pajari_drop/main_env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Loading data as single file

In [2]:
months = {'june': 6, 'july': 7}

vendors_df = pd.DataFrame()
important_cols = ['locode', 'phrase', 'average_msv']

for month, month_id in tqdm(months.items()):
    data_file = f'data/{month}.csv'
    month_data = pd.read_csv(data_file, error_bad_lines=False)[important_cols]
    month_data.rename(columns={'average_msv': f'avg_msv_{month}'}, inplace=True)
    if not len(vendors_df):
        vendors_df = month_data
    else:
        vendors_df = vendors_df.merge(month_data, on=['locode', 'phrase'])

  0%|          | 0/2 [00:00<?, ?it/s]

b'Skipping line 244321: expected 19 fields, saw 20\nSkipping line 244385: expected 19 fields, saw 20\n'
b'Skipping line 237738: expected 19 fields, saw 20\nSkipping line 237802: expected 19 fields, saw 20\n'


In [3]:
vendors_df.head(5)

,locode,phrase,avg_msv_june,avg_msv_july
0,AE,0 annual fee credit card,10,10
1,AE,0% apr credit cards,10,10
2,AE,0% credit card offers,10,10
3,AE,0% credit cards,20,20
4,AE,0% interest credit cards,50,50


In [4]:
def round_to_next_existing(val, msv_unique_values):
    '''used for impressions only'''
    if val: # Not 0 and not None  
        if len(msv_unique_values[msv_unique_values>val])>0: # val is less than the largest existing value 
            return msv_unique_values[msv_unique_values>=val][0]
        else: # val is larger than the largest existing value 
            return msv_unique_values[-1]
    else:  # 0 or None  
        return val

def categorize(rounded_x, msv_unique_values):
    return np.where(msv_unique_values == rounded_x)[0][0]

msv_unique_values = get_unique_msv()


Get unique existing msv values...
[        0        10        20        30        40        50        70
        90       110       140       170       210       260       320
       390       480       590       720       880      1000      1300
      1600      1900      2400      2900      3600      4400      5400
      6600      8100      9900     12100     14800     18100     22200
     27100     33100     40500     49500     60500     74000     90500
    110000    135000    165000    201000    246000    301000    368000
    450000    550000    673000    823000   1000000   1220000   1500000
   1830000   2240000   2740000   3350000   4090000   5000000   6120000
   7480000   9140000  11100000  13600000  16600000  20400000  24900000
  30400000  37200000  45500000  55600000  83100000 101000000 124000000
 151000000 185000000 226000000]


In [5]:
avg_msv_cols = [col_name for col_name in list(vendors_df) if 'avg_msv' in col_name]
# replacing 0 to 10 because we need to follow 2 rules:
# 1. that if one month msv = 0 and another = 0 so ratio has to be 1
# 2. that if one month msv = 0 and another > 0 so first month has to be 10
vendors_df[avg_msv_cols] = vendors_df[avg_msv_cols].replace(0, 10)

#### Categorizing columns

In [6]:
avg_msv_cat_cols = []
for col in avg_msv_cols:
    print(col)
    round_col = f'{col}_round'
    cat_col = f'{col}_cat'
    
    vendors_df[round_col] = vendors_df[col].apply(func=round_to_next_existing, args=(msv_unique_values, ))
    vendors_df[cat_col] = vendors_df[round_col].apply(func=categorize, args=(msv_unique_values, ))
    avg_msv_cat_cols.append(cat_col)

avg_msv_june
avg_msv_july


####  calculating and choosing max ratio

In [7]:
for cat_col in avg_msv_cat_cols:
    current_month = cat_col.split('_')[2]
    current_month_id = months[current_month]
    prev_month = [month for month, month_id in months.items() if month_id == current_month_id - 1]
    if not prev_month:
        continue
    else:
        prev_month = prev_month[0]

    prev_month_cat_col = [cat_col for cat_col in avg_msv_cat_cols if prev_month in cat_col][0]

    change_df = pd.DataFrame()
    change_df['inc'] = vendors_df[cat_col] / vendors_df[prev_month_cat_col]
    change_df['dec'] = vendors_df[prev_month_cat_col] / vendors_df[cat_col]

    change_col = f'{current_month}_{prev_month}_cat_change_ratio'

    vendors_df[change_col] = change_df.max(axis=1) 

In [11]:
gold_ratio_col = 'july_june_cat_change_ratio'
gold_ratio_values = vendors_df.loc[vendors_df[gold_ratio_col] > 3].copy()
# gold_mu = gold_ratio_values[gold_ratio_col].mean(axis=0)
# gold_std = gold_ratio_values[gold_ratio_col].std(axis=0)

gold_mu = 8.428
gold_std = 6.814

gold_exp = 0.04

In [12]:
print(f'mu = {gold_mu}, std = {gold_std}, exp = {gold_exp}')

mu = 8.428, std = 6.814, exp = 0.04


In [13]:
ratio_cols = [col_name for col_name in list(vendors_df) if 'ratio' in col_name]

In [17]:
prob_df = pd.DataFrame()
anomalies_count = list()
for col_name in ratio_cols:
    high_ratio_values = vendors_df.loc[vendors_df[col_name] > 3].copy()
    high_ratio_values[f'prob_{col_name}'] = high_ratio_values[col_name].apply(lambda x: np.prod(norm.pdf(x, 
                                                                                                         gold_mu, 
                                                                                                         gold_std)))
    data = col_name.split('_')
    month_to = data[0]
    month_from = data[1]
    anomalies = high_ratio_values[high_ratio_values[f'prob_{col_name}'] < gold_exp]
    print(month_from, month_to, len(anomalies), f'{round(len(anomalies) / len(high_ratio_values), 4)}% of distributed')
    anomalies_count.append(len(anomalies))

june july 136 0.108% of distributed


In [15]:
vendors_df

,locode,phrase,avg_msv_june,avg_msv_july,avg_msv_june_round,avg_msv_june_cat,avg_msv_july_round,avg_msv_july_cat,july_june_cat_change_ratio
0,AE,0 annual fee credit card,10,10,10,1,10,1,1.0
1,AE,0% apr credit cards,10,10,10,1,10,1,1.0
2,AE,0% credit card offers,10,10,10,1,10,1,1.0
3,AE,0% credit cards,20,20,20,2,20,2,1.0
4,AE,0% interest credit cards,50,50,50,5,50,5,1.0
...,...,...,...,...,...,...,...,...,...
9817191,ZW HRE,wework,40,40,40,4,40,4,1.0
9817192,ZW HRE,wework office space,10,10,10,1,10,1,1.0
9817193,ZW HRE,wework space,10,10,10,1,10,1,1.0
9817194,ZW HRE,workspace for rent,10,10,10,1,10,1,1.0


### eps findings

In [31]:
ratio_test_col = 'june_may_cat_change_ratio'
test_high_ratio = vendors_df.loc[vendors_df[ratio_test_col] > 3].copy()
prob_test_col = f'prob_{ratio_test_col}' # col for choosing eps param

test_mu = test_high_ratio[ratio_test_col].mean()
test_std = test_high_ratio[ratio_test_col].std()

test_high_ratio[prob_test_col] = test_high_ratio[ratio_test_col].apply(lambda x: np.prod(norm.pdf(x, test_mu, test_std)))

In [32]:
for eps in 1e-1**(np.arange(1, 10, 0.1)):
    selected_data = test_high_ratio[test_high_ratio[prob_test_col] < eps]
    print ('eps = {}, perc_outliers = {:.3f}%, ratio min = {:.3f}, ratio std = {:.3f}'.format(eps,
                                                                                              len(selected_data) / len(vendors_df) * 100,
                                                                                              selected_data[ratio_test_col].min(),
                                                                                              selected_data[ratio_test_col].std()))


eps = 0.1, perc_outliers = 0.035%, ratio min = 3.050, ratio std = 6.814
eps = 0.07943282347242814, perc_outliers = 0.035%, ratio min = 3.050, ratio std = 6.814
eps = 0.0630957344480193, perc_outliers = 0.035%, ratio min = 3.050, ratio std = 6.814
eps = 0.0501187233627272, perc_outliers = 0.017%, ratio min = 3.050, ratio std = 9.288
eps = 0.03981071705534969, perc_outliers = 0.005%, ratio min = 14.500, ratio std = 8.186
eps = 0.031622776601683764, perc_outliers = 0.004%, ratio min = 16.000, ratio std = 8.228
eps = 0.025118864315095774, perc_outliers = 0.003%, ratio min = 17.333, ratio std = 8.341
eps = 0.01995262314968877, perc_outliers = 0.003%, ratio min = 19.000, ratio std = 8.392
eps = 0.01584893192461111, perc_outliers = 0.002%, ratio min = 19.500, ratio std = 8.505
eps = 0.01258925411794165, perc_outliers = 0.002%, ratio min = 20.500, ratio std = 8.582
eps = 0.009999999999999981, perc_outliers = 0.002%, ratio min = 21.500, ratio std = 8.657
eps = 0.007943282347242798, perc_outlier

In [ ]:
eps = 0.03
anomaly = high_ratio_values[high_ratio_values[prob_test_col] < eps]

In [ ]:
anomaly.to_csv('anomaly_0.03_exploration.csv', index=False)

In [ ]:
len(anomaly) / len(vendors_df) * 100

In [ ]:
len(anomaly)

In [ ]:
anomaly.june_may_cat_change_ratio.describe()

In [ ]:
anomaly